In [ ]:
####################################
#ENVIRONMENT SETUP

In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py

from tqdm import tqdm

In [ ]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [ ]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, SlurmJobArray_Class, DataManager_Class

In [ ]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"3_Project_Algorithms","1_Domain_Profiles"))
from CLASSES_DomainProfiles import DomainProfiles_Class, DomainProfiles_DataLoading_Class

In [ ]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"3_Project_Algorithms","2_Tracking_Algorithms"))
from CLASSES_TrackingAlgorithms import TrackingAlgorithms_DataLoading_Class

In [ ]:
#IMPORT FUNCTIONS
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
import FUNCTIONS_Variable_Calculation
from FUNCTIONS_Variable_Calculation import *

In [ ]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=1)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="Tracking_Algorithms", dataName="Eulerian_CLTracking",
                                dtype='float32',codeSection = "Project_Algorithms")

In [ ]:
#JOB ARRAY SETUP
UsingJobArray=True

def GetNumJobs(res):
    if res=='1km':
        num_jobs=20
    elif res=='250m': 
        num_jobs=100
    return num_jobs
num_jobs = GetNumJobs(ModelData.res)
SlurmJobArray = SlurmJobArray_Class(total_elements=ModelData.Ntime, num_jobs=num_jobs, UsingJobArray=UsingJobArray)
start_job = SlurmJobArray.start_job; end_job = SlurmJobArray.end_job

def GetNumElements():
    num_elements = np.arange(ModelData.Ntime)[start_job:end_job]
    return num_elements
num_elements = GetNumElements()

In [ ]:
##############################################
#MODEL AND ALGORITHM NUMERICAL PARAMETERS

In [ ]:
kms=np.argmax(ModelData.xh-ModelData.xh[0] >= 1) #finds how many x grids is 1 km
dx=int(ModelData.xh[1]-ModelData.xh[0]) #grid resolution (in km)

In [ ]:
#setting convergence threshold
if ModelData.res=='1km':
    conv_thresh=1.0/1000
elif ModelData.res=="250m":
    conv_thresh=1.5/1000

In [ ]:
##############################################
#DATA LOADING FUNCTIONS

In [ ]:
def GetConvergence(t):
    timeString = ModelData.timeStrings[t]
    varName = 'convergence'
    convergence = CallVariable(ModelData, DataManager, timeString, varName)
    return convergence

In [ ]:
##############################################
#ALGORITHM FUNCTIONS

In [ ]:
#SBZ Convergence Line Search Algorithm (levels are seperate) (python version 3.10.9) (All Max Algorithm)
def FindMaxIndices(yconv):
    """find indices of local maxima above threshold."""

     #takes dconv/dx
    ddx = (yconv[1:  ] - yconv[0:-1]) / (2 * dx)
     
    #finds local max where dconv/dx sign changes
    signs = np.sign(ddx)
    signs_diff=np.diff(signs)
    local_maxes=np.where((signs_diff != 0) & (signs_diff < 0))[0]+1 #make sure +1 is here (it corrects the location of the derivative)
    local_maxes=local_maxes[np.where(yconv[local_maxes]>conv_thresh)] #check if convergence is greater than convergence threshold (1s-1)
    local_maxes=local_maxes[(local_maxes>50*kms)&(local_maxes<len(ModelData.xf)-50*kms)] #removes maxes that are with 50 km of y boundary
    return local_maxes

In [ ]:
#Finds all local maximums (from Calculus) along each y level for a specific z level (~0.28km in this case)
def FindLocalMaxes(conv_z,yind, second_round=False):
    """
    #Find_Local_Maxes Function
    #(1) At a single time and z level, runs through each y-level
    #(2) At each y-level, takes the x-derivative
    #(3) Take sign(x_derivative)
    #(4) Take diff(x_derivative)
    #(5) Max is located one index to the right of where derivative changes from positive to negative or diff is +1
    #[(6) Optional: the algorithm can run a second time over the leftover maxes after removing previous maxes from temporary variable]
    """
    
    #indexes convergence in y
    yconv=conv_z[yind,:]
    
    #finds local max where dconv/dx sign changes
    local_maxes = FindMaxIndices(yconv)

    #second round maxes (not 100% necessary, only if missing many convergence maximums that are visually there)
    if second_round == True:
        yconv2=yconv.copy()
        yconv2[local_maxes]=0
        
        #takes dconv/dx
        local_maxes2 = FindMaxIndicies(f=yconv2)
    
        #combine the two
        local_maxes=np.concatenate((local_maxes,local_maxes2))
    return local_maxes

In [ ]:
##############################################
#RUNNING FUNCTIONS

In [ ]:
def LayerMax(conv): #finds max convergence along y for multiple z location (5 is good)
    
    #initializing data
    Nz = int(np.where(ModelData.zh <= 0.775)[0][-1]) + 1 #only include levels less than 1 km
    maxConvergence_X=np.full((Nz, ModelData.Nyh, ModelData.Nxh), -1, dtype=np.int16)
    
    #running for all z levels
    for zlev in range(Nz):
        #Taking Convergence of current timesftep
        conv_z=conv[zlev,:,:] #current z level for convergence

        for yind in range(ModelData.Nyh): #plot maximums for each row

            #finds all local maxes
            local_maxes=FindLocalMaxes(conv_z,yind) #convergence threshold (in 1/s)
            
            #storing data
            maxConvergence_X[zlev,yind,local_maxes] = local_maxes
    return maxConvergence_X

def GetAvgConvergence(conv):
    # #OPTION ONE
    # z_level = np.where(ModelData.zh<=1)[0][-1]
    # return np.mean(conv[0:z_level+1], axis=(1)) #worst option
    
    # #OPTION TWO
    # z_level = np.where(ModelData.zh<=500/1e3)[0][-1]
    # return np.mean(conv[z_level],axis=0) #second best option
    
    #OPTION THREE
    return np.mean(conv, axis=(0,1)) #BEST OPTION

In [ ]:
def RunAlgorithm():
    for t in tqdm(num_elements, desc="Processing timesteps"):
        # if t % 1 == 0: print(f'Processing timestep {t}/{len(data["time"])}')

        #getting convergence at time t
        conv=GetConvergence(t)

        # Compute maxConvergence_X for this timestep (z,y,x)
        maxConvergence_X = LayerMax(conv)

        avgConvergence = GetAvgConvergence(conv)

        Dictionary = {"maxConvergence_X": maxConvergence_X,
                      "avgConvergence": avgConvergence}

        # Directly write into HDF5 dataset at index t
        timeString = ModelData.timeStrings[t]
        TrackingAlgorithms_DataLoading_Class.SaveData(ModelData,DataManager, Dictionary, timeString)

In [ ]:
#############################################
#RUNNING

In [ ]:
RunAlgorithm()

In [ ]:
#############################################
#TESTING

In [ ]:
#testing max convergence average propagation

In [ ]:
#LOADING CL MAXS FROM CL TRACKING ALGORITHM
def Get_AvgConvergence(t):

    timeString = ModelData.timeStrings[t]
    outputDataDirectory=os.path.normpath(os.path.join(DataManager.outputDataDirectory,"..","Eulerian_CLTracking"))
    Dictionary = TrackingAlgorithms_DataLoading_Class.LoadData(ModelData, DataManager, timeString,
                     dataName="Eulerian_CLTracking",outputDataDirectory=outputDataDirectory)
    whereCL = Dictionary["avgConvergence"]
    return whereCL

In [ ]:
where_lst=[]
for count,t in enumerate(np.arange(0,132,1)):
    d=Get_AvgConvergence(t=t)
    dmax = np.max(d)
    where = np.where(d==dmax); 
    if t==0:
        where_lst.append(np.nan)
    else:
        where_lst.append(where[0].item())

    # #plotting
    # plt.plot(d)
    # plt.scatter(where,dmax,label=t)
# plt.legend()

plt.plot(np.arange(len(where_lst)),where_lst)
plt.xlabel("time-index")
plt.ylabel("convergence max x-index")
plt.title("Propagation of Max Convergence (e.g. SBF)")